In [1]:
import pandas as pd
from pandas import Series, DataFrame # 最常用的类

In [2]:
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.set_printoptions(precision=4, suppress=True) # numpy 不要使用科学计数法，小数4位

## 分类数据

`pandas`的`Categorical`类型
- `cut`和`qcut`函数的返回类型

### 背景和目标

In [3]:
values = pd.Series(['apple', 'orange', 'apple',
                    'apple'] * 2)
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [4]:
values.unique()

array(['apple', 'orange'], dtype=object)

In [5]:
values.value_counts()

apple     6
orange    2
dtype: int64

许多的数据系统（用于数据入库， 统计计算，或其他用途）已经开发出专门的方法，**用重复的值来表示数据**， 以便有效的存储和计算

在数据入库的操作中，使用所谓的“维度表”是这一种最佳实践
- 包含不同值
- 将**主要观测值**存储为引用维度表的**整数键**

In [6]:
values = pd.Series([0,1,0,0] * 2)
dim = pd.Series(['apple', 'orange'])

In [7]:
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [8]:
dim

0     apple
1    orange
dtype: object

In [9]:
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

按照整数展现的方式我们称为“**分类**”或“**字典编码**”，**不同值的数组可以被称为数据的类别， 字典或层级，`level`**

- 用整数来展示的方式，称为**分类，`categorical`， 字典编码方式**
- 存储不同值的数组 叫做 `categories`， 字典或者数据的`level`
- 分类完成后的整数叫做 `category code`

在做数据分析时，**分类展示会产生显著的性能提升**，也可以在类别上进行转换同时不改变代码

#### pandas中的Categorical类型

 pandas中的Categorical类型，用于承载**基于整数的类别展示**或**编码**的数据

In [10]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2
N = len(fruits)
df = pd.DataFrame({'fruit': fruits,
                   'basket_id': np.arange(N),
                   'count': np.random.randint(3, 15, size=N),
                   'weight': np.random.uniform(0, 4, size=N)},
                  columns=['basket_id', 'fruit', 'count', 'weight'])
df

,basket_id,fruit,count,weight
0,0,apple,5,3.858058
1,1,orange,8,2.612708
2,2,apple,4,2.995627
3,3,apple,7,2.614279
4,4,apple,12,2.990859
5,5,orange,8,3.845227
6,6,apple,5,0.033553
7,7,apple,4,0.425778


In [11]:
fruit_cat = df['fruit'].astype('category')

In [12]:
fruit_catfruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [15]:
type(fruit_cat), type(fruit_cat.index), type(fruit_cat.values) 

(pandas.core.series.Series,
 pandas.core.indexes.range.RangeIndex,
 pandas.core.arrays.categorical.Categorical)

In [17]:
c = fruit_cat.values
c 

['apple', 'orange', 'apple', 'apple', 'apple', 'orange', 'apple', 'apple']
Categories (2, object): ['apple', 'orange']

In [18]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [19]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [20]:
df['fruit'] = df['fruit'].astype('category')

In [21]:
df

,basket_id,fruit,count,weight
0,0,apple,5,3.858058
1,1,orange,8,2.612708
2,2,apple,4,2.995627
3,3,apple,7,2.614279
4,4,apple,12,2.990859
5,5,orange,8,3.845227
6,6,apple,5,0.033553
7,7,apple,4,0.425778


In [26]:
my_category = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
my_category

['foo', 'bar', 'baz', 'foo', 'bar']
Categories (3, object): ['bar', 'baz', 'foo']

In [27]:
my_category.codes

array([2, 0, 1, 2, 0], dtype=int8)

In [24]:
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]

In [25]:
my_category = pd.Categorical.from_codes(codes, categories)
my_category

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

In [28]:
my_category = pd.Categorical.from_codes(codes, categories, ordered=True)
my_category

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

### 使用Categorical对象进行计算

在pandas中使用Categories与非编码版本相比，**整体上是一致的**， Pandas的某些部分，比如**`groupby`函数，在于Categories对象协作工作时，性能更好**，还有一些函数可以利用`ordered`标识

In [34]:
np.random.seed(12345)
draws = np.random.randn(1000)
draws[:5]

array([-0.2047,  0.4789, -0.5194, -0.5557,  1.9658])

In [35]:
bins = pd.qcut(draws, 4)
bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

In [36]:
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
bins

['Q2', 'Q3', 'Q2', 'Q2', 'Q4', ..., 'Q3', 'Q2', 'Q1', 'Q3', 'Q4']
Length: 1000
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [37]:
bins.categories

Index(['Q1', 'Q2', 'Q3', 'Q4'], dtype='object')

In [39]:
bins.codes[:10]

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

In [40]:
bins = pd.Series(bins, name='quartile')
bins

0      Q2
1      Q3
2      Q2
3      Q2
4      Q4
       ..
995    Q3
996    Q2
997    Q1
998    Q3
999    Q4
Name: quartile, Length: 1000, dtype: category
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [41]:
result = pd.Series(draws).groupby(bins).agg( ['count', 'max', 'min'])

In [42]:
result

,count,max,min
quartile,,,
Q1,250,-0.685484,-2.949343
Q2,250,-0.010115,-0.683066
Q3,250,0.628894,-0.010032
Q4,250,3.927528,0.634238


In [43]:
result.reset_index()

,quartile,count,max,min
0,Q1,250,-0.685484,-2.949343
1,Q2,250,-0.010115,-0.683066
2,Q3,250,0.628894,-0.010032
3,Q4,250,3.927528,0.634238


#### 使用分类获得更高性能

如果在特定的数据集上做了大量的分析，**将数据转换为分类数据可以产生大幅度的性能提升**， *DataFrame中一列的分类版本通常也会明显使用更少的内存*

In [44]:
N= 10000000

In [45]:
draws = pd.Series(np.random.randn(N))

In [48]:
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))

In [50]:
categories = labels.astype('category')

In [51]:
labels.memory_usage()

80000128

In [52]:
categories.memory_usage()

10000320

分类转换当然不是免费的，但是他是**一次性操作**，使用分类对象进行GroupBy操作明显更快，这是因为**底层算法基于整数代码的数组而不是字符串**

### 分类的方法

Series包含的分类数据拥有一些特殊方法，这些方法类似于Series.str的特殊字符串，**这些方法提供了快捷访问类别和代码的方式**

特殊属性`cat`提供了分类方法的访问

In [53]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)
cat_s = s.astype('category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [54]:
cat_s.cat

In [56]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [57]:
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [60]:
cat_s.values.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [61]:
cat_s.values.codes

array([0, 1, 2, 3, 0, 1, 2, 3], dtype=int8)

In [62]:
actual_categories = ['a', 'b', 'c', 'd', 'e']
cat_s2 = cat_s.cat.set_categories(actual_categories)
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): ['a', 'b', 'c', 'd', 'e']

In [64]:
cat_s2.unique()

['a', 'b', 'c', 'd']
Categories (4, object): ['a', 'b', 'c', 'd']

In [65]:
cat_s2.value_counts()

d    2
c    2
b    2
a    2
e    0
dtype: int64

In [66]:
cat_s2.isin(['b', 'c'])

0    False
1     True
2     True
3    False
4    False
5     True
6     True
7    False
dtype: bool

In [67]:
cat_s2.cat.remove_unused_categories()

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

#### 创建用于建模的虚拟变量

当时用统计数据或机器学习工具时，通常会**将分类数据转换为虚拟变量**，也称为`one-hot`.这回产生一个DataFrame，**每个不同的类别都是他的一列，这些列包含一个特定类别出现的次数，否则为0**

In [68]:
cats = pd.Series(list('abcd'), dtype='category')
cats

0    a
1    b
2    c
3    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [70]:
pd.get_dummies(cats)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1


## 高阶GroupBy应用

### 分组转换和“展开”GroupBy

分组操作中可以用`apply`方法用于执行转换操作，`transform`与`apply`相似，但是会对你**可以使用的函数中类加上跟过限制**
- 可以产生一个标示量，并广播到个分组的尺寸数据中
- 可以产生一个与输入分组尺寸相同的对象
- 不改变它的输入

In [71]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                   'value': np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [73]:
df.groupby('key').value

In [74]:
df.groupby('key').value.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [75]:
df.groupby('key').mean()

,value
key,
a,4.5
b,5.5
c,6.5


In [77]:
df.groupby('key').transform('mean')

,value
0,4.5
1,5.5
2,6.5
3,4.5
4,5.5
5,6.5
6,4.5
7,5.5
8,6.5
9,4.5


In [78]:
df.groupby('key').transform(lambda x: x.mean())

,value
0,4.5
1,5.5
2,6.5
3,4.5
4,5.5
5,6.5
6,4.5
7,5.5
8,6.5
9,4.5


In [79]:
df.groupby('key').transform(lambda x: x**2)

,value
0,0.0
1,1.0
2,4.0
3,9.0
4,16.0
5,25.0
6,36.0
7,49.0
8,64.0
9,81.0


In [80]:
df.groupby('key').transform(lambda x: x.rank(ascending=False))

,value
0,4.0
1,4.0
2,4.0
3,3.0
4,3.0
5,3.0
6,2.0
7,2.0
8,2.0
9,1.0


In [81]:
def normalize(x):
    return (x - x.mean()) / x.std()

In [82]:
df.groupby('key').transform(normalize)

,value
0,-1.161895
1,-1.161895
2,-1.161895
3,-0.387298
4,-0.387298
5,-0.387298
6,0.387298
7,0.387298
8,0.387298
9,1.161895


In [83]:
df.groupby('key').apply(normalize)

,key,value
0,NaN,-1.161895
1,NaN,-1.161895
2,NaN,-1.161895
3,NaN,-0.387298
4,NaN,-0.387298
5,NaN,-0.387298
6,NaN,0.387298
7,NaN,0.387298
8,NaN,0.387298
9,NaN,1.161895


In [90]:
(df['value'] -df.groupby('key').value.transform('mean')) /df.groupby('key').value.transform('std')

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

### 分组的时间重新采样

`resample`方法在语义上是一种**基于时间分段的分组操作**

In [91]:
N = 15
times = pd.date_range('2017-05-20 00:00', freq='1min', periods=N)
df = pd.DataFrame({'time': times,
                   'value': np.arange(N)})
df

,time,value
0,2017-05-20 00:00:00,0
1,2017-05-20 00:01:00,1
2,2017-05-20 00:02:00,2
3,2017-05-20 00:03:00,3
4,2017-05-20 00:04:00,4
5,2017-05-20 00:05:00,5
6,2017-05-20 00:06:00,6
7,2017-05-20 00:07:00,7
8,2017-05-20 00:08:00,8
9,2017-05-20 00:09:00,9


In [94]:
df.set_index('time').resample('5min').count()

,value
time,
2017-05-20 00:00:00,5
2017-05-20 00:05:00,5
2017-05-20 00:10:00,5


In [96]:
df2 = pd.DataFrame({'time': times.repeat(3),
                    'key': np.tile(['a', 'b', 'c'], N),
                    'value': np.arange(N * 3.)})
df2

,time,key,value
0,2017-05-20 00:00:00,a,0.0
1,2017-05-20 00:00:00,b,1.0
2,2017-05-20 00:00:00,c,2.0
3,2017-05-20 00:01:00,a,3.0
4,2017-05-20 00:01:00,b,4.0
...,...,...,...
40,2017-05-20 00:13:00,b,40.0
41,2017-05-20 00:13:00,c,41.0
42,2017-05-20 00:14:00,a,42.0
43,2017-05-20 00:14:00,b,43.0


In [110]:
time_key = pd.Grouper(freq='5T')

In [111]:
time_key

TimeGrouper(freq=<5 * Minutes>, axis=0, sort=True, closed='left', label='left', how='mean', convention='e', origin='start_day')

In [112]:
df2.set_index('time').groupby(['key', time_key]).count()

value
key time                      
a   2017-05-20 00:00:00      5
    2017-05-20 00:05:00      5
    2017-05-20 00:10:00      5
b   2017-05-20 00:00:00      5
    2017-05-20 00:05:00      5
    2017-05-20 00:10:00      5
c   2017-05-20 00:00:00      5
    2017-05-20 00:05:00      5
    2017-05-20 00:10:00      5

In [113]:
df2.set_index('time').groupby(['key', time_key]).sum()

value
key time                      
a   2017-05-20 00:00:00   30.0
    2017-05-20 00:05:00  105.0
    2017-05-20 00:10:00  180.0
b   2017-05-20 00:00:00   35.0
    2017-05-20 00:05:00  110.0
    2017-05-20 00:10:00  185.0
c   2017-05-20 00:00:00   40.0
    2017-05-20 00:05:00  115.0
    2017-05-20 00:10:00  190.0

## 方法链技术